# HttpCommand

> Ignorance more frequently begets confidence than does knowledge: it is those who know little, not those who know much, who so positively assert that this or that problem will never be solved by science. --_Charles Darwin_

Ah yes, the web. I'm sure you've heard of it. Dyalog has a nifty http client library built in, called `HttpCommand`. In order to make us of it, we first need to load it up:

In [95]:
]load HttpCommand
⎕IO ← 0
{}⎕SE.UCMD'box on -s=min -t=tree -f=on'
{}⎕SE.UCMD'rows on'

#.HttpCommand

There is a handy web service delivering random Kanye West quotations we can put to good use, `kanye.rest`, to demonstrate this. Kanye as a Service?

In [52]:
url ← 'https://api.kanye.rest/'
⎕←z←HttpCommand.Get url

[rc: 0 | msg: "" | HTTP Status: 200 "OK" | ⍴Data: 95]

What did we get back? Let's look in the headers to start with:

In [56]:
{(⍵[;0]∊⊂'Content-Type')⌿⍵} z.Headers

┌→───────────┬────────────────┐
↓Content-Type│application/json│
└───────────→┴───────────────→┘

We know JSON; good. Let's unpack that.

In [57]:
j←⎕JSON z.Data

In [60]:
]map j  ⍝ list the fields

#.[JSON object]
· ~ quote

So the payload we're interested in is in the `quote` field of the `j` namespace:

In [61]:
j.quote

I love UZI. I be saying the same thing about Steve Jobs. I be feeling just like UZI

Thanks for that, Kanye. Here's the seminal [Gold Digger](https://open.spotify.com/track/3QHPHLAkYV5cQBUYs6rowx?si=7JnJeGWiT6aGCAXr1YlRIg), feat. Jamie Foxx. If you're easily offended by colorful language, that's probably not a choon for you.

Anyway. Back to APL. Let's look at a more complex data set. The Cloudant database `https://skruger.cloudant.com/airaccidents` contains a large data set drawn from the FAA, listing air accident reports. [Cloudant](https://www.ibm.com/cloud/cloudant) is a db as a service running the open source [CouchDB](https://couchdb.apache.org/) database, a JSON-over-HTTP distributed document store. Let's pull a few documents from it and see what they look like.

We're going to hit the `_all_docs` endpoint, but as this is a large database, we want to only fetch one documents, so we want to pass the parameters `limit=1` and `include_docs=true` on the URL.

In [72]:
url←'https://skruger.cloudant.com/airaccidents/_all_docs'
(params←⎕NS'').(include_docs limit)←'true' 1

In [94]:
z←HttpCommand.Get url params

We know it's going to be JSON, as everything in CouchDB is JSON.

In [74]:
j←⎕JSON z.Data
]map j

#.[JSON object] 
· ~ offset rows total_rows

For the `_all_docs` API endpoint, the data is returned under `rows`, and if we set the `include_docs` parameter, each of those entries will have a `doc` field, containing the document itself. Let's look at the first one.

In [75]:
]map j.rows[0].doc

#.[JSON object].[JSON object].[JSON object] 
· ~ Country Latitude Location Longitude Make Model Schedule _id _rev 
· ~ ⍙Accident⍙32⍙Number ⍙Aircraft⍙32⍙Category ⍙Aircraft⍙32⍙Damage 
· ~ ⍙Airport⍙32⍙Code ⍙Airport⍙32⍙Name ⍙Air⍙32⍙Carrier ⍙Amateur⍙32⍙Built 
· ~ ⍙Broad⍙32⍙Phase⍙32⍙of⍙32⍙Flight ⍙Engine⍙32⍙Type ⍙Event⍙32⍙Date 
· ~ ⍙Event⍙32⍙Id ⍙FAR⍙32⍙Description ⍙Injury⍙32⍙Severity 
· ~ ⍙Investigation⍙32⍙Type ⍙Number⍙32⍙of⍙32⍙Engines ⍙Publication⍙32⍙Date 
· ~ ⍙Purpose⍙32⍙of⍙32⍙Flight ⍙Registration⍙32⍙Number ⍙Report⍙32⍙Status 
· ~ ⍙Total⍙32⍙Fatal⍙32⍙Injuries ⍙Total⍙32⍙Minor⍙32⍙Injuries 
· ~ ⍙Total⍙32⍙Serious⍙32⍙Injuries ⍙Total⍙32⍙Uninjured ⍙Weather⍙32⍙Condition

Yuck. What _is_ that crap? So the JSON field names aren't valid APL names, meaning Dyalog had to mangle them when converting to namespaces. We _can_ read them like that if we want to, for example

In [76]:
j.rows[0].doc.⍙Event⍙32⍙Date

10/10/1982

The other option is to convert the data to an array instead:

In [77]:
⎕←j←⎕JSON⍠'M' ⊢ z.Data

┌→┬──────────────────────┬──────────────────────────────────┬─┐
↓0│ │0 │1│
├─┼─────────────────────⊖┼~────────────────────────────────⊖┼─┤
│1│total_rows │68389 │3│
├─┼─────────────────────→┼~─────────────────────────────────┼─┤
│1│offset │0 │3│
├─┼─────────────────────→┼~─────────────────────────────────┼─┤
│1│rows │0 │2│
├─┼─────────────────────→┼~────────────────────────────────⊖┼─┤
│2│ │0 │1│
├─┼─────────────────────⊖┼~────────────────────────────────⊖┼─┤
│3│id │42788bb50806d9cc7770d46953000c99 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│3│key │42788bb50806d9cc7770d46953000c99 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│3│value │0 │1│
├─┼─────────────────────→┼~────────────────────────────────⊖┼─┤
│4│rev │1-0f7d34e40be0c3d4db805cf52c4adf42│4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│3│doc │0 │1│
├─┼─────────────────────→┼~────────────────────────────────⊖┼─┤
│4│_id │42788bb50806d9cc7770d46953000c99 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│_rev │1-0f7d34e40be0c3d4db805cf52c4adf42│4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Air Carrier │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│4│Aircraft Category │Airplane │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Injury Severity │Fatal(1) │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Event Id │20020917X05139 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Country │United States │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Airport Name │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│4│Total Fatal Injuries │1 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Total Minor Injuries │1 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Total Uninjured │0 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Latitude │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│4│Amateur Built │No │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Event Date │10/10/1982 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Report Status │Probable Cause │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Airport Code │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│4│FAR Description │Part 91: General Aviation │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Schedule │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│4│Publication Date │10/10/1983 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Longitude │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│4│Make │BELLANCA │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Model │7GCBC │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Engine Type │Reciprocating │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Total Serious Injuries│0 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Purpose of Flight │Personal │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Broad Phase of Flight │CLIMB │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Weather Condition │VMC │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Aircraft Damage │Substantial │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Accident Number │SEA83FYM01 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Location │1/4NM S. OF PEO, OR │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Registration Number │N57457 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Number of Engines │1 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│4│Investigation

Which is actually quite suitable for this data -- the documents are completely flat. The documents themselves are at "depth 4", as indicated by the first column.

The database has a few handy indexes, too, which in CouchDB-speak is called _views_. Let's look at a couple of those. The first view allows us to fetch documents based on the make of aircraft. Here's the first entry in the index where the make of the plane involved was a Cessna:

In [82]:
url←'https://skruger.cloudant.com/airaccidents/_design/make/_view/by-make'
(params←⎕NS'').(limit reduce key)←1 'false' '"Cessna"'
z←HttpCommand.Get url params
⎕←j←⎕JSON⍠'M' ⊢ z.Data

[rc: 0 | msg: "" | HTTP Status: 200 "OK" | ⍴Data: 116]

This is a materialised view, keyed on make. The view iteself contains no particularly interesting information beyond the document id and the "value" 1. We can fetch this document by the id:

In [84]:
url←'https://skruger.cloudant.com/airaccidents/5b97c6d78b17b37ceff620baf9657693'
z←HttpCommand.Get url
⎕←j←⎕JSON⍠'M' ⊢ z.Data

[rc: 0 | msg: "" | HTTP Status: 200 "OK" | ⍴Data: 893]

┌→┬──────────────────────┬──────────────────────────────────┬─┐
↓0│ │0 │1│
├─┼─────────────────────⊖┼~────────────────────────────────⊖┼─┤
│1│_id │5b97c6d78b17b37ceff620baf9657693 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│_rev │1-8d2e3c5096718be0236ef51ff7e0f153│4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Air Carrier │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Aircraft Category │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Injury Severity │Non-Fatal │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Event Id │20080611X00825 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Country │United States │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Airport Name │PORTLAND-TROUTDALE │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Total Fatal Injuries │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Total Minor Injuries │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Total Uninjured │1 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Latitude │45.549444 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Amateur Built │No │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Event Date │05/24/2008 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Report Status │Probable Cause │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Airport Code │TTD │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│FAR Description │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Schedule │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Publication Date │06/30/2008 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Longitude │-122.401389 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Make │Cessna │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Model │172P │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Engine Type │Reciprocating │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Total Serious Injuries│ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Purpose of Flight │Instructional │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Broad Phase of Flight │ │4│
├─┼─────────────────────→┼─────────────────────────────────⊖┼─┤
│1│Weather Condition │VMC │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Aircraft Damage │Substantial │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Accident Number │LAX08CA156 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Location │Troutdale, OR │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Registration Number │N62348 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Number of Engines │1 │4│
├─┼─────────────────────→┼─────────────────────────────────→┼─┤
│1│Investigation Type │Accident │4│
└─┴─────────────────────→┴─────────────────────────────────→┴─┘

but perhaps more interesting is that we can do aggregations if we switch on the reduce part in the view:

In [93]:
url←'https://skruger.cloudant.com/airaccidents/_design/make/_view/by-make'
(params←⎕NS'').(reduce key)←'true' '"Cessna"'
z←HttpCommand.Get url params
j←⎕JSON z.Data
j.rows[0].value

7728

Reductions in CouchDB views are similar to reduces in APL. All we did there was a `+/` over the values in the view, which as we saw earlier was a "1".

Now we're running the risk of making this about the CouchDB API, but this is quite an interesting data set. I made a [video](https://www.youtube.com/watch?v=2SXPMCvuTQA) a long time ago about it and how to process the data with map-reduce on CouchDB, and this was all inspired by a very old [blog post](https://blog.cloudant.com/2011/01/13/mapreduce-from-the-basics-to-the-actually-useful.html) from Cloudant founder, [Mike Miller](https://www.linkedin.com/in/mlmilleratmit). 